In [1]:
import json
import requests
import webbrowser
import base64

client_id = 'YOUR_CLIENET_ID'
client_secret = 'YOUR_CLIENT_SECRET'
redirect_url = 'REDIRECT_URL'
scope = 'offline_access accounting.transactions'
b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

def XeroFirstAuth():
    # 1. Send a user to authorize your app
    auth_url = ('''https://login.xero.com/identity/connect/authorize?''' +
                '''response_type=code''' +
                '''&client_id=''' + client_id +
                '''&redirect_uri=''' + redirect_url +
                '''&scope=''' + scope +
                '''&state=123''')
    
    print(auth_url)
#     webbrowser.open_new(auth_url)
    
    # 2. Users are redirected back to you with a code
    auth_res_url = input('What is the response URL? ')
    start_number = auth_res_url.find('code=') + len('code=')
    end_number = auth_res_url.find('&scope')
    auth_code = auth_res_url[start_number:end_number]
    print(auth_code)
    print('\n')
    
    # 3. Exchange the code
    exchange_code_url = 'https://identity.xero.com/connect/token'
    response = requests.post(exchange_code_url, 
                            headers = {
                                'Authorization': 'Basic ' + b64_id_secret
                            },
                            data = {
                                'grant_type': 'authorization_code',
                                'code': auth_code,
                                'redirect_uri': redirect_url
                            })
    json_response = response.json()
    print(json_response)
    print('\n')
    
    # 4. Receive your tokens
    return [json_response['access_token'], json_response['refresh_token']]

In [ ]:
XeroFirstAuth()

In [3]:
# 5. Check the full set of tenants you've been authorized to access
def XeroTenants(access_token):
    connections_url = 'https://api.xero.com/connections'
    response = requests.get(connections_url,
                           headers = {
                               'Authorization': 'Bearer ' + access_token,
                               'Content-Type': 'application/json'
                           })
    json_response = response.json()
    print(json_response)
    
    for tenants in json_response:
        json_dict = tenants
    return json_dict['tenantId']

In [4]:
# 6.1 Refreshing access tokens
def XeroRefreshToken(refresh_token):
    token_refresh_url = 'https://identity.xero.com/connect/token'
    response = requests.post(token_refresh_url,
                            headers = {
                                'Authorization' : 'Basic ' + b64_id_secret,
                                'Content-Type': 'application/x-www-form-urlencoded'
                            },
                            data = {
                                'grant_type' : 'refresh_token',
                                'refresh_token' : refresh_token
                            })
    json_response = response.json()
    print(json_response)
    
    new_refresh_token = json_response['refresh_token']
    rt_file = open('refresh_token.txt', 'w')
    rt_file.write(new_refresh_token)
    rt_file.close()
    
    return [json_response['access_token'], json_response['refresh_token']]

In [12]:
def refresh_token():
    old_tokens = XeroFirstAuth()
    new_tokens = XeroRefreshToken(old_tokens[1])
    return new_tokens

In [37]:
def get_online_invoice(invoice_id, token=None, tenant_id=None):
    if token == None:
        refresh_token()
    if tenant_id == None:
        xero_tenant_id = XeroTenants(new_tokens[0])
    
    get_url = 'https://api.xero.com/api.xro/2.0/Invoices/{0}/OnlineInvoice'.format(invoice_id)
    response = requests.get(get_url,
                           headers = {
                               'Authorization': 'Bearer ' + token[0],
                               'Xero-tenant-id': tenant_id,
                               'Accept': 'application/json'
                           })
    try:
        json_response = response.json()
    except:
        print(response)
    return json_response['OnlineInvoices'][0]['OnlineInvoiceUrl']

In [ ]:
new_tokens = refresh_token()
xero_tenant_id = XeroTenants(new_tokens[0])

In [ ]:
from csv import reader
import time

online_invoices_arr = []
with open('invoices.csv', 'r') as doc:
    csv_reader = reader(doc, delimiter=",")
    next(csv_reader)
    for invoice_id in csv_reader:
        online_invoice_id = get_online_invoice(invoice_id[0], new_tokens, xero_tenant_id)
        time.sleep(5)
        online_invoices_arr.append(invoice_id)
        
        online_invoices = open("online-invoices.csv", 'a')
        
        online_invoices.write(f'\n{invoice_id[0]+"",""+online_invoice_id}')
        online_invoices.close()